In [6]:
# import os
# os.environ['HF_TOKEN'] = 'hf_..'
base_data_path = '/content/ROCOv2-radiology/source_dataset'
train_file = f'{base_data_path}/train_captions.csv'
validation_file = f'{base_data_path}/valid_captions.csv'
train_concepts_file = f'{base_data_path}/train_concepts_manual.csv'
validation_concepts_file = f'{base_data_path}/valid_concepts_manual.csv'
test_file = f'{base_data_path}/test_captions.csv'
test_concepts_file = f'{base_data_path}/test_concepts_manual.csv'
dataset_name = 'rajatgupta99924/roco-mri-v2'

### Login to Hugging Face

In [3]:
# from huggingface_hub import set_environment_variable
# import os

# HF_TOKEN = os.environ.get('HF_TOKEN')

# if HF_TOKEN is None:
#     raise ValueError("Please set your Hugging Face token as an environment variable 'HF_TOKEN'")

# set_environment_variable("HF_TOKEN", HF_TOKEN)
# print("Hugging Face token successfully set.")

from huggingface_hub import login
import os

HF_TOKEN = ""

if os.environ.get('HF_TOKEN') is not None:
  HF_TOKEN = os.environ.get('HF_TOKEN')
  print(f"Hugging Face token found in environment variable")
try:
  import google.colab
  from google.colab import userdata
  if (userdata.get('HF_TOKEN') is not None) and (HF_TOKEN == ""):
    HF_TOKEN = userdata.get('HF_TOKEN')
  else:
    raise ValueError("Please set your Hugging Face token in the user data panel, or pass it as an environment variable")
except ModuleNotFoundError:
  if HF_TOKEN is None:
    raise ValueError("Please set your Hugging Face token in the user data panel, or pass it as an environment variable")

login(
  token=HF_TOKEN,
  add_to_git_credential=True
)

ImportError: cannot import name 'set_environment_variable' from 'huggingface_hub' (/usr/local/lib/python3.12/dist-packages/huggingface_hub/__init__.py)

In [7]:
from huggingface_hub import login
import os

HF_TOKEN = os.environ.get('HF_TOKEN')

if HF_TOKEN is None:
    raise ValueError("Please set your Hugging Face token as an environment variable 'HF_TOKEN'")

login(
  token=HF_TOKEN
)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
!pip install --upgrade huggingface_hub

In [ ]:
!git clone https://github.com/sctg-development/ROCOv2-radiology.git

In [8]:
from datasets import load_dataset, Dataset, Image as HFImage, concatenate_datasets
import datasets
from PIL import Image
import pandas as pd
import io
import os

# load image in the DataFrame
def load_image(image_id, image_split='train'):
    image_path = os.path.join(f'/content/ROCOv2-radiology/source_dataset/{image_split}', f"{image_id}.jpg")
    image_jpg= Image.open(image_path)
    image_bytes = io.BytesIO()
    image_jpg.save(image_bytes, format='PNG')  # Save as PNG
    # Replace PIL image with a new PNG image created from the bytes
    return image_bytes.getvalue()

# Function to apply load_cui with progress tracking
def apply_with_progress(df, func, column, nb, image_path='train'):
    result = []
    for i, value in enumerate(df[column]):
        result.append(func(value, image_path))
        if (i + 1) % nb == 0:
            print(f"Processed {i + 1} rows")
    return result

### Create the train split

In [9]:
# Load the CUI CSV file into a pandas DataFrame
train_concept_unique_identifier_df = pd.read_csv(train_concepts_file)

# load CUI to the train_df DataFrame by looking up the CUI in the concept_unique_dentifier_df
# concept_unique_dentifier_df is a DataFrame that contains the mapping between the image ID and the CUIs
def load_train_cui(image_id, image_path='train'):
    cuis = train_concept_unique_identifier_df[train_concept_unique_identifier_df['ID'] == image_id]['CUIs']
    split = str(cuis.values[0]).split(';')
    return split

# Load a CSV file into a pandas DataFrame
train_df = pd.read_csv(train_file)
train_df.rename(columns={'ID': 'image_id', 'Caption': 'caption'}, inplace=True)
# Filter the DataFrame to include only rows where the caption contains "MRI" or "magnetic resonance imaging"
train_df = train_df[train_df['caption'].str.lower().str.contains('mri|magnetic resonance imaging', na=False)]
train_df['image'] = apply_with_progress(train_df, load_image, 'image_id',100, 'train')
train_df = train_df[['image', 'image_id', 'caption']]
train_df['cui'] = apply_with_progress(train_df, load_train_cui, 'image_id',1000, 'train')
train_dataset = Dataset.from_pandas(train_df).cast_column("image", HFImage())
train_dataset.save_to_disk('train_dataset')
# train_dataset.push_to_hub(dataset_name)

Processed 100 rows
Processed 200 rows
Processed 300 rows
Processed 400 rows
Processed 500 rows
Processed 600 rows
Processed 700 rows
Processed 800 rows
Processed 900 rows
Processed 1000 rows
Processed 1100 rows
Processed 1200 rows
Processed 1300 rows
Processed 1400 rows
Processed 1500 rows
Processed 1600 rows
Processed 1700 rows
Processed 1800 rows
Processed 1900 rows
Processed 2000 rows
Processed 2100 rows
Processed 2200 rows
Processed 2300 rows
Processed 2400 rows
Processed 2500 rows
Processed 2600 rows
Processed 2700 rows
Processed 2800 rows
Processed 2900 rows
Processed 3000 rows
Processed 3100 rows
Processed 3200 rows
Processed 3300 rows
Processed 3400 rows
Processed 3500 rows
Processed 3600 rows
Processed 3700 rows
Processed 3800 rows
Processed 3900 rows
Processed 4000 rows
Processed 4100 rows
Processed 4200 rows
Processed 4300 rows
Processed 4400 rows
Processed 4500 rows
Processed 4600 rows
Processed 4700 rows
Processed 4800 rows
Processed 4900 rows
Processed 5000 rows
Processed

Saving the dataset (0/3 shards):   0%|          | 0/5538 [00:00<?, ? examples/s]

In [11]:
from datasets import load_from_disk
import random

# Load the training dataset from disk
train_dataset_loaded = load_from_disk('train_dataset')

# Get 10 random indices
random_indices = random.sample(range(len(train_dataset_loaded)), 10)

# Print the captions for the random entries
print("Randomly selected captions from the training dataset:")
for i, index in enumerate(random_indices):
    caption = train_dataset_loaded[index]['caption']
    print(f"{i+1}: {caption}")

Randomly selected captions from the training dataset:
1: Magnetic resonance imaging after 1 month of radiotherapy, showing a dimensional decrease of the mass (22 × 15 mm) (diameters shown by red lines)
2: T2-weighted MRI of the abdomen showing bilateral adrenal masses. The left mass measures 4.3 cm and the right mass measures 2.8 cm in greatest dimension.
3: Maxillofacial and neck magnetic resonance imaging (axial view): presence of a voluminous solid mass at the base of the tongue with oropharyngeal space reduction.
4: MRI of the arm showed heterogenous mass on the medial aspect of the left arm (blue arrow) and matted axillary lymph nodes (red arrow).
5: Axial head MRI.
6: Axial T1 with Gadolinium MRI showing no evidence of recurrence at 10-year follow-up.
7: Cerebellar atrophy in a chronic phenytoin user. Coronal T2-weighted MRI sequence showing marked atrophy of the cerebellum, disproportionate to that observed in the rest of the brain parenchyma, in an 18-year-old patient who had b

### Create the validation split

In [10]:
# Load the CUI CSV file into a pandas DataFrame
valid_concept_unique_identifier_df = pd.read_csv(validation_concepts_file)

def load_valid_cui(image_id, image_path='train'):
    cuis = valid_concept_unique_identifier_df[valid_concept_unique_identifier_df['ID'] == image_id]['CUIs']
    split = str(cuis.values[0]).split(';')
    return split

valid_df = pd.read_csv(validation_file)
valid_df.rename(columns={'ID': 'image_id', 'Caption': 'caption'}, inplace=True)
# Filter the DataFrame to include only rows where the caption contains "MRI" or "magnetic resonance imaging"
valid_df = valid_df[valid_df['caption'].str.lower().str.contains('mri|magnetic resonance imaging', na=False)]
valid_df['image'] = apply_with_progress(valid_df, load_image, 'image_id',100, 'valid')
valid_df = valid_df[['image', 'image_id', 'caption']]
valid_df['cui'] = apply_with_progress(valid_df, load_valid_cui, 'image_id',1000, 'valid')
valid_dataset = Dataset.from_pandas(valid_df).cast_column("image", HFImage())
valid_dataset.save_to_disk('valid_dataset')

Processed 100 rows
Processed 200 rows
Processed 300 rows
Processed 400 rows
Processed 500 rows
Processed 600 rows
Processed 700 rows
Processed 800 rows
Processed 900 rows


Saving the dataset (0/1 shards):   0%|          | 0/954 [00:00<?, ? examples/s]

### Create the test split

In [12]:
# Load the CUI CSV file into a pandas DataFrame
test_concept_unique_identifier_df = pd.read_csv(test_concepts_file)

def load_test_cui(image_id, image_path='train'):
    cuis = test_concept_unique_identifier_df[test_concept_unique_identifier_df['ID'] == image_id]['CUIs']
    split = str(cuis.values[0]).split(';')
    return split


test_df = pd.read_csv(test_file)
test_df.rename(columns={'ID': 'image_id', 'Caption': 'caption'}, inplace=True)
# Filter the DataFrame to include only rows where the caption contains "MRI" or "magnetic resonance imaging"
test_df = test_df[test_df['caption'].str.lower().str.contains('mri|magnetic resonance imaging', na=False)]
test_df['image'] = apply_with_progress(test_df, load_image, 'image_id',100, 'test')
test_df = test_df[['image', 'image_id', 'caption']]
test_df['cui'] = apply_with_progress(test_df, load_test_cui, 'image_id',1000, 'test')
test_dataset = Dataset.from_pandas(test_df).cast_column("image", HFImage())
test_dataset.save_to_disk('test_dataset')

Processed 100 rows
Processed 200 rows
Processed 300 rows
Processed 400 rows
Processed 500 rows
Processed 600 rows
Processed 700 rows
Processed 800 rows
Processed 900 rows


Saving the dataset (0/1 shards):   0%|          | 0/918 [00:00<?, ? examples/s]

### Push the datasets to the Hugging Face hub

In [13]:
train_dataset.push_to_hub(dataset_name, split='train')
valid_dataset.push_to_hub(dataset_name, split='validation')
test_dataset.push_to_hub(dataset_name, split='test')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ? shards/s]

Map:   0%|          | 0/1846 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/19 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :   3%|2         | 6.29MB /  241MB            

README.md:   0%|          | 0.00/558 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rajatgupta99924/roco-mri-v2/commit/bdbd4d48b1e4aad264533bcfe206e0f5c7800d92', commit_message='Upload dataset', commit_description='', oid='bdbd4d48b1e4aad264533bcfe206e0f5c7800d92', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/rajatgupta99924/roco-mri-v2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='rajatgupta99924/roco-mri-v2'), pr_revision=None, pr_num=None)